## [5. AQ]

#### Recursive Algorithm

We developed a first algotithm to solve the task. The idea behind it is to use the recursion in the following way:
-  we start with initial score $S$ and a vector of marks $p = [p_1, p_2, \dots, p_n]$
1. if $p$ has only one element then return it
2. otherwise for each element $p_i$ of $p$ repeat from 1. but considering as new score $S=p_i $, and also update the vector $p$ following the rules in the text of the exercise. Save every of this $n$ maximum score values in an array and return the maximum.

This algorithm causes a lot of memory usage and has an high complexity due to the high number of functions that are called in cascades. 
Below we report the algorithm:


In [2]:
def get_score_rec(S: int, p: list) -> int:
  
  # Store the length of the marks vector
  n = len(p)
  # Initializate an empty vector to store every possible maximum score
  # for every possible choice of the 'next' exam
  # It will be of length n
  values = []
  
  # Check if p has only one element
  # In that case the maximum score you can get is p[0]
  if n == 1:
    return p[0]
  else:
    
    # If p has more than one element then we cycle throughout all the possible
    # choices of the 'next' exam taken
    for i in range(n):
      
      # Store the 'next' exam taken as the new personal score
      S_tmp = p[i]
      
      # If he takes an 'easy' exam
      if S > p[i]:
        
        # Compute the difference between the score and the exam
        diff = S - p[i]
        
        # Remove from the list the exam and update all the other exams
        # In this case all the other exams become more difficult
        q = [p[j] + diff for j in range(n) if j != i]
        
      # In the case he takes an 'hard' exam
      elif S < p[i]:
        
        # Compute the difference between the exam and the score
        diff = p[i] - S
        
        # Remove from the list the exam and update all the other exams
        # In this case all the other exams become easier
        q = [p[j] - diff for j in range(n) if j != i]
        
      # If the exam has the same value as the personal score
      else:
        
        # Simply remove that exame from the vector 
        q = [p[j] for j in range(n) if j != i]
      
      # Then call recursively the function get_score_rec
      # passing as arguments the new score and the new vector
      # of length n - 1.
      # Store the maximum result, for that case, in the values array
      values.append(get_score_rec(S_tmp, q.copy()))
      
    # Now get the maximum element from all the possible n choices
    # of the 'next' exam and return it
    max_value = max(values)
    return max_value
  
# # Example of usage
# S = 25
# p = [18, 24, 21, 32, 27]
# 
# highest_score = get_score_rec(S, p)
# print(f'Highest mark: {highest_score}')     

This function has a very big time complexity. Before computing the worst case running time $T(n)$ we report here the code with annotated the running time of each operation.
```
def get_score_rec(S: int, p: list) -> int:
  
  n = len(p)                                            # O(1)
  values = []                                           # O(1)
  
  if n == 1:                                            # O(1)
    return p[0]                                         # O(1)
  else:
    for i in range(n):                                  # O(n)
      S_tmp = p[i]                                      # * O(1)
      if S > p[i]:                                      # * O(1)
        diff = S - p[i]                                 # * O(1)
        q = [p[j] + diff for j in range(n) if j != i]   # * O(n)
      elif S < p[i]:                                    # * O(1)
        diff = p[i] - S                                 # * O(1)
        q = [p[j] - diff for j in range(n) if j != i]   # * O(n)
      else:
        q = [p[j] for j in range(n) if j != i]          # * O(n)
      values.append(get_score_rec(S_tmp, q.copy()))     # * T(n-1)
    max_value = max(values)                             # O(n)
    return max_value                                    # O(1)  
```

Summing up all the comments about the running time, we see that $T(n)$ is recursively defined as
$\quad T(n) \sim
\begin{cases}
\mathcal{O}(1) & \text{ if }n = 1\\
\mathcal{O}(n^2) + n\ T(n-1) & \text{ if }n > 1\\ 
\end{cases}
$

It can be also written as follow:
$$ 
\begin{align*}
T(n) & \sim \mathcal{O}(n^2) + n\ T(n-1) \sim \mathcal{O}(n^2) + n\big(\mathcal{O}(n^2) + (n-1) T(n-2)\big)\sim \mathcal{O}(n^3) + n(n-1)T(n-2)\\
& \sim \mathcal{O}(n^3) + n(n-1)\big(\mathcal{O}(n^2) + n\ T(n-3)\big) \sim \mathcal{O}(n^4) + n(n-1)(n-2)T(n-3)\\
& \sim\dots\sim \mathcal{O}(n^{n+1}) + n!\ T(1)\sim \mathcal{O}(n^{n+1})
\end{align*}
$$
So $T(n)\sim \mathcal{O}(n^{n+1})$ and this algorithm has exponential running time.

#### Optimized Algorithm

Since $\mathcal{O}(n^{n+1})$ is a very high time complexity, we looked for an optimized algorithm to solve the problem. In particular we found that:
- consider the vector of all marks $p = [p_1, p_2, \dots, p_n]$ to be ordered such that $p_1\leq p_2\leq \dots\leq p_n$:
  - if $n$ is odd then the final mark does not depend on the initial score $S$ and is:  $\displaystyle\sum_{j=\frac{n-1}2+1}^{n}p_i - \sum_{i=1}^{\frac{n-1}2}p_j$
  - if $n$ is even then the final mark is:  $S + \displaystyle\sum_{j=\frac{n}2+1}^{n}p_i - \sum_{i=1}^{\frac{n}2}p_j$

It's simple to show that if $n$ is odd then the final mark does not depend on $S$. In fact Federico, the ficticious student, has to make $n-1$ choices for the exams (the last is dependent on those previous choices). Every $2$ choices $S$ is first added and then subtracted, so it disappears. But $n$ is odd, so $n-1$ is divisible by $2$ and that means that $S$ does not appear in the final score. 
We say that "Every $2$ choices $S$ is first added and then subtracted" because we can (WLOG) look at the following example where $n=3$:
- we have $S$ and $p = [p_1, p_2, p_3]$ 
- choose one of the three arbitrarily, WLOG $p_3$:
  - if $S \geq p_3$ then
    - $S^{(1)} = p_3$ and $p = [p_1 + S - p_3, p_2 + S - p_3]$ 
    
      then WLOG we pick $p_2 + S - p_3$
    - if $S^{(1)} \geq p_2 + S - p_3$ 

      then $S^{(2)} = p_2 + S - p_3$ and $p = [p_1 + S - p_3 + (S^{(1)} - (p_2 + S - p_3))] = [p_1 + S -p_3 +(p_3 - p_2 - S + p_3)] = [p_3 + p_1 - p_2]$ and the final score is $p_3 + p_1 - p_2$ and does not depend on $S$
    - if $S^{(1)} < p_2 + S - p_3$ 

      then $S^{(2)} = p_2 + S - p_3$ and $p = [p_1 + S - p_3 - ((p_2 + S - p_3) - S^{(1)})] = [p_1 + S -p_3 - (p_2 + S - p_3 - p_3)] = [p_3 + p_1 - p_2]$ and the final score is $p_3 + p_1 - p_2$ and does not depend on $S$
  - if $S < p_3$ then the final mark is the same, but we omit to write the calculus and all the subcases.

From this also follows that in the case of $n$ even then the initial personal score $S$ is summed to the final maximum score because here we have one more choice, that is equivalent to add $S$ to the final result computed considering $S^{(0)} = 0$ and $p = [p_1,p_2,\dots,p_n]$.


Now we prove our optimized algorithm using strong induction on the length $n$ of the vector of exams. 

If $n=1$ the vector of exams is $[p_1]$. Then the final score is $p_1$ and is the maximum obtainable.

If $n=2$ the vector of exams is $[p_1, p_2]$. Let us analyze all the possibilities:
- if Federico chooses $p_1$ as the first exam:
  - if $S > p_1$: final score = $p_2 + S - p_1$
  - if $S \leq p_1$: final score = $p_2 - p_1 + S$
- if Federico chooses $p_2$ as the first exam:
  - if $S > p_2$: final score = $p_1 + S - p_2$
  - if $S \leq p_2$: final score = $p_1 - p_2 + S$
  
So there are only two possibilities for the final score: $p_2-p_1 + S$ to $p_1 - p_2 + S$. Since $p_2 \geq p_1$ then the maximum final score will be $p_2-p_1 + S$ and follows the rules of our algorithm. 

By inductive hypothesis we have that the algorithm is right $\forall k<n$. Our discussion is divided into two cases, $n$ odd and $n$ even. 

If $n$ is odd we WLOG assume  that $S^{(0)} = 0$, because we have shown that the final score does not depend on the initial score. We want the last step (when we have only two exams left to choose from) to be to increase the final score, and to do this we must necessarily start by increasing all the exams by as much as possible. We want to increase them all, so Federico must choose as the first exam $p_n$ which has the largest value. By doing this we can increase the remaining exams, which become $p_i^{(1)}= p_i + p_n$ and $S^{(1)} =p_n$.
Again, we want to increase the exams values by as much value as possible. Since $S^{(1)} =p_n$ then Federico will have to choose $S^{(2)} =p_1^{(1)}$ since $p_n-p_1^{(1)}$ is the greatest possible difference between our exams. The remaining exams will then become $p_i^{(2)} = p_i^{(1)} + (p_n - p_1^{(1)}) = (p_i + p_n)+(p_n-p_1-p_n) = p_i + p_n - p_1$ and the score $S^{(2)} = p_n - p_1$. But now we can use the inductive hypothesis since we have a vector of votes of length less than $n$. So the final maximum score will be $$S^{(final)} = p_n - p_1 + p_{n-1}-p_2 + \dots + p_{\frac{n+3}2} - p_{\frac{n-1}2} + p_{\frac{n+1}2} = \displaystyle\sum_{j=\frac{n-1}2+1}^{n}p_i - \sum_{i=1}^{\frac{n-1}2}p_j$$

If $n$ is even the final score is the score we get considering $S^{(0)} = 0$, plus the original initial score. So we can assume $S^{(0)} = 0$. The treatment is analogous to that in the case of odd $n$ but starting this time by considering $p_1$, and concluding again with the inductive hypothesis.


Below is the implemented optimized algorithm. 

In [3]:
def get_score_optimized(S: int, p: list) -> int:
  
  # Store the length of the marks vector
  n = len(p)
  
  # Sort the vector of marks in ascending order
  p = sorted(p)
  
  # First summation:
  # If n is even, then the first summation is the sum of the first n/2 elements
  # If n is odd, then the first summation is the sum of the first (n-1)/2 elements
  jnk1 = sum([p[i] for i in range(0, n // 2)])
  
  # Second summation:
  # If n is even, then the second summation is the sum of the last n/2 elements
  # If n is odd, then the second summation is the sum of the last (n+1)/2 elements
  jnk2 = sum([p[i] for i in range(n // 2, n)]) 
  
  # Use the formulas explained in the report to compute the maximum score
  if n % 2 == 0:  
    return(S + jnk2 - jnk1)
    
  elif n % 2 == 1:
    return(jnk2 - jnk1)

Here we comment the code with the running time complexity of each operation.

```
def get_score_optimized(S: int, p: list) -> int:
  n = len(p)                                      # O(1)
  
  p = sorted(p)                                   # O(n*log(n))
  jnk1 = sum([p[i] for i in range(0, n // 2)])    # 
  jnk2 = sum([p[i] for i in range(n // 2, n)])    # O(n) (considering also the previous line) 
  
  if n % 2 == 0:                                  # O(1)
    return(S + jnk2 - jnk1)                       # O(1)
    
  elif n % 2 == 1:                                # O(1)
    return(jnk2 - jnk1)                           # O(1)
```
  
Summing up all the comments about the running time, we see can easily see that $T(n)$ is basically the cost of the sorting operation. If we consider, for example, merge-sort or quick-sort (that in practice is usually faster than merge-sort) we can say that $T(n)\sim \mathcal{O}(n\log n)$.  

In particular $T(n)\sim \Theta(n \log n)$ because every solution must order the values in the array, so our is an optimal solution. 

Finally, to show that both our algorithms works, we wrote a little script to test them of the sample inputs provided by the text of the exercise. Below you can see the results.

In [4]:
S = []
p = []

  
S.append(8)
p.append([5, 7, 1])

S.append(25)
p.append([18, 24, 21, 32, 27])


S.append(30)
p.append([13, 27, 41, 59, 28, 33, 39, 19, 52, 48, 55, 79])

for Score, exams in zip(S, p):
  print(f'Initial score: {Score}\nExams: {exams}')
  highest_score1 = get_score_rec(Score, exams)
  print(f'Recursive: {highest_score1}')  
  
  highest_score2 = get_score_optimized(Score, exams)
  print(f'Optimized: {highest_score2}')  
  print('-'*30)

Initial score: 8
Exams: [5, 7, 1]
Recursive: 11
Optimized: 11
------------------------------
Initial score: 25
Exams: [18, 24, 21, 32, 27]
Recursive: 44
Optimized: 44
------------------------------
Initial score: 30
Exams: [13, 27, 41, 59, 28, 33, 39, 19, 52, 48, 55, 79]
Recursive: 205
Optimized: 205
------------------------------


#### ChatGPT algorithm

We tried several times to query ChatGPT, and it was difficult to get it to cover the problem. Unfortunately, he only succeeded in giving us a suboptimal algorithm (which we report below) with running time complexity equal to $\mathcal{O}(n!\ n^2)$. 
ChatGPT was unable to provide a corrected algorithm with lower running time.

In [ ]:
## ChatGPT implementation

from itertools import permutations

def calculate_final_score(initial_score, marks):
    S = initial_score
    marks = list(marks)  # Convert tuple to a mutable list

    for i, mark in enumerate(marks):
        if mark < S:  # Easy exam
            for j in range(len(marks)):
                if j != i:
                    marks[j] += S - mark  # Adjust marks for easy exam
            S = mark  # Update Federico's score
        else:  # Hard exam
            for j in range(len(marks)):
                if j != i:
                    marks[j] -= mark - S  # Adjust marks for hard exam
            S = mark  # Update Federico's score

    return S

def find_maximum_final_score(initial_score, marks):
    max_final_score = initial_score
    max_permutation = marks

    # Generate all permutations of the exams
    all_permutations = permutations(marks)

    # Calculate final score for each permutation
    for permutation in all_permutations:
        final_score = calculate_final_score(initial_score, permutation)
        if final_score > max_final_score:
            max_final_score = final_score
            max_permutation = permutation

    return max_final_score, max_permutation

# Example usage
initial_score = 25
exam_marks = (18, 24, 21, 32, 27)

max_score, best_permutation = find_maximum_final_score(initial_score, exam_marks)
print("Maximum final score:", max_score)
print("Best permutation of exams:", best_permutation)


Maximum final score: 44
Best permutation of exams: (24, 18, 32, 21, 27)
